# Upper Triangular Matrix Vector Multiply Routines

This notebook walks you through how to implement $ y := U x + y $ where $ U $ is upper triangular.

## Getting started

We will use some functions that are part of our laff library (of which this function will become a part) as well as some routines from the FLAME API (Application Programming Interface) that allows us to write code that closely resembles how we typeset algorithms using the FLAME notation.  These functions are imported with `include` and `using` statements.

## Algorithm that takes dot products

<image src="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/432_Trmvp_un_unb_var1.png" alt="Alternative matrix-vector multiplication via dot products algorithm" width="50%">

## The `Tmvmult_un_unb_var1!( U, x, y )` routine

This routine, given upper triangular $ U \in \mathbb{R}^{n \times n} $, $ x \in \mathbb{R}^n $, and $ y \in \mathbb{R}^n $, computes $ y := U x + y $.  The "_un_" in the name of the routine indicates this is the "upper, no transpose" matrix-vector multiplication.  

The specific laff functions we will use are 
<ul>
<li> <code> laff.dots!( x, y, alpha ) </code> which computes $ \alpha := x^T y + \alpha $.  </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [6]:
include("../flame.jl")
using .flame
include("../laff/laff.jl")
using .laff

function Tmvmult_un_unb_var1!(U, x, y)

    UTL, UTR,
    UBL, UBR  = flame.part_2x2(U,
                               0, 0, "TL")

    xT,
    xB  = flame.part_2x1(x,
                         0, "TOP")

    yT,
    yB  = flame.part_2x1(y,
                         0, "TOP")

    while size(UTL, 1) < size(U, 1)

        U00,  u01,       U02,
        u10t, upsilon11, u12t,
        U20,  u21,       U22   = flame.repart_2x2_to_3x3(UTL, UTR,
                                                         UBL, UBR,
                                                         1, 1, "BR")

        x0, 
        chi1,
        x2    = flame.repart_2x1_to_3x1(xT,
                                        xB,
                                        1, "BOTTOM")

        y0,
        psi1,
        y2    = flame.repart_2x1_to_3x1(yT,
                                        yB,
                                        1, "BOTTOM")

        #------------------------------------------------------------#

        laff.dots!( upsilon11, chi1, psi1 )
        laff.dots!( u12t, x2, psi1 )

        #------------------------------------------------------------#

        UTL, UTR,
        UBL, UBR  = flame.cont_with_3x3_to_2x2(U00,  u01,       U02,
                                               u10t, upsilon11, u12t,
                                               U20,  u21,       U22,
                                               "TL")

        xT,
        xB  = flame.cont_with_3x1_to_2x1(x0,
                                         chi1,
                                         x2,
                                         "TOP")

        yT,
        yB  = flame.cont_with_3x1_to_2x1(y0,
                                         psi1,
                                         y2,
                                         "TOP")

    end
    flame.merge_2x1!(yT,
                    yB, y)
end


Tmvmult_un_unb_var1! (generic function with 1 method)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [2]:
U = rand(4, 4)
x = rand(4)
y = rand(4)
yold = rand(4)

# Notice that U is not upper triangular.  We will only use the upper triangular part.

println("U before = ")
U

U before = 


4×4 Array{Float64,2}:
 0.538827  0.293987   0.307639   0.388301
 0.171535  0.180947   0.73742    0.398915
 0.726384  0.86314    0.0795606  0.879939
 0.689384  0.0524589  0.552817   0.949237

In [3]:
println("x before = ")
x

x before = 


4-element Array{Float64,1}:
 0.6005267519720021
 0.6839212247260473
 0.5917169411570158
 0.8137994656063363

In [4]:
println("y before = ")
y

y before = 


4-element Array{Float64,1}:
 0.8389488496326893  
 0.014657870452313482
 0.035244365885907625
 0.836457594917462   

In [7]:
laff.copy!( y, yold )   # save the original vector y

Tmvmult_un_unb_var1!( U, x, y )

println( "y after =" )
y

y after =


4-element Array{Float64,1}:
 1.8616270659557501
 0.8993920646748057
 0.7984154466808487
 1.6089461292686118

In [12]:
using LinearAlgebra # To grab the `UpperTriangular` function
println( "y - ( UpperTriangular( U ) * x + yold ) = " ) #np.triu makes the matrix upper triangular
y - ( UpperTriangular( U ) * x + yold )

y - ( UpperTriangular( U ) * x + yold ) = 


4-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0

Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.

## Algorithm that uses axpys

<image src="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/432_Trmvp_un_unb_var2.png" alt="Alternative matrix-vector multiplication via dot products algorithm" width="50%">

## The `Tmvmult_un_unb_var2!( U, x, y )` routine

This routine, given upper triangular $ U \in \mathbb{R}^{n \times n} $, $ x \in \mathbb{R}^n $, and $ y \in \mathbb{R}^n $, computes $ y := U x + y $.  The "_un_" in the name of the routine indicates this is the "upper triangular, no transpose" matrix-vector multiplication.  

The specific laff functions we will use are 
<ul>
<li> <code> laff.axpy!( alpha, x, y ) </code> which computes $ y := \alpha x +  y  $.  </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [13]:
include("../flame.jl")
using .flame
include("../laff/laff.jl")
using .laff

function Tmvmult_un_unb_var2!(U, x, y)

    UTL, UTR,
    UBL, UBR  = flame.part_2x2(U, 
                               0, 0, "TL")

    xT, 
    xB  = flame.part_2x1(x, 
                         0, "TOP")

    yT, 
    yB  = flame.part_2x1(y, 
                         0, "TOP")

    while size(UTL, 1) < size(U, 1)

        U00,  u01,       U02,  
        u10t, upsilon11, u12t, 
        U20,  u21,       U22   = flame.repart_2x2_to_3x3(UTL, UTR, 
                                                         UBL, UBR, 
                                                         1, 1, "BR")

        x0,   
        chi1, 
        x2    = flame.repart_2x1_to_3x1(xT, 
                                        xB, 
                                        1, "BOTTOM")

        y0,   
        psi1, 
        y2    = flame.repart_2x1_to_3x1(yT, 
                                        yB, 
                                        1, "BOTTOM")

        #------------------------------------------------------------#

        laff.axpy!( chi1, u01, y0 )
        laff.axpy!( chi1, upsilon11, psi1 )

        #------------------------------------------------------------#

        UTL, UTR, 
        UBL, UBR  = flame.cont_with_3x3_to_2x2(U00,  u01,       U02,  
                                               u10t, upsilon11, u12t, 
                                               U20,  u21,       U22,  
                                               "TL")

        xT, 
        xB  = flame.cont_with_3x1_to_2x1(x0,   
                                         chi1, 
                                         x2,   
                                         "TOP")

        yT, 
        yB  = flame.cont_with_3x1_to_2x1(y0,   
                                         psi1, 
                                         y2,   
                                         "TOP")

    end
    flame.merge_2x1!(yT,
                    yB, y)
end


Tmvmult_un_unb_var2! (generic function with 1 method)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [14]:
U = rand(4, 4)
x = rand(4)
y = rand(4)
yold = rand(4)

# Notice that U is not upper triangular.  We will only use the upper triangular part.

println("U before = ")
U

U before = 


4×4 Array{Float64,2}:
 0.940508  0.928183  0.658398  0.517847
 0.516931  0.139879  0.199248  0.619878
 0.590316  0.191372  0.638425  0.341745
 0.949035  0.235357  0.956007  0.60394 

In [15]:
println("x before = ")
x

x before = 


4-element Array{Float64,1}:
 0.6941117636315033 
 0.47407864746824635
 0.6082685049081167 
 0.8991810042996795 

In [16]:
println("y before = ")
y

y before = 


4-element Array{Float64,1}:
 0.7521584670587655 
 0.22550803805197628
 0.9043506075146779 
 0.44489870922803876

In [17]:
laff.copy!( y, yold )   # save the original vector y

Tmvmult_un_unb_var2!( U, x, y )

println( "y after =" )
y

y after =


4-element Array{Float64,1}:
 2.7111290111615487
 0.9704003577734339
 1.5999751545698269
 0.9879499724894265

In [18]:
using LinearAlgebra # To grab the `UpperTriangular` function
println( "y - ( UpperTriangular( U ) * x + yold ) = " ) #np.triu makes the matrix upper triangular
y - ( UpperTriangular( U ) * x + yold )

y - ( UpperTriangular( U ) * x + yold ) = 


4-element Array{Float64,1}:
  0.0                   
 -1.1102230246251565e-16
  0.0                   
  0.0                   

Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.